### Loading data and libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing, model_selection,feature_extraction

In [2]:
from sklearn.model_selection import RandomizedSearchCV

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from nlp_utils.model import train_model

In [6]:
data_folder = 'data/'

In [7]:
## using the cleaned files
train_data = pd.read_csv(data_folder+'train_clean.csv'); print(train_data.shape)
test_data = pd.read_csv(data_folder+'test_clean.csv'); print(test_data.shape)

(7613, 6)
(3263, 5)


In [8]:
train_data.head()

,id,keyword,location,text,target,text_clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order cali...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [9]:
train_data[train_data['target'] == 1].sample()['text'].values[0]

"Thanks a lot roadworks men cos a tube strike wasn't disruptive enough so having to walk the desolate route from Tottenham to ....."

In [10]:
# train_data.location.value_counts()

In [11]:
sample_submission = pd.read_csv(data_folder+'sample_submission.csv')
# sample_submission.head()

### Word Embeddings

#### Pre-processing

In [12]:
from tensorflow.keras.preprocessing import text, sequence

In [13]:
## creating a tokenizer
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(train_data['text'])

In [14]:
word_index = tokenizer.word_index

In [15]:
len(word_index)

22700

In [16]:
# convering tweets to sequences and padding them
train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(train_data['text']), maxlen=70,padding='post')
test_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(test_data['text']), maxlen=70,padding='post')

In [17]:
train_seq_x

array([[ 119, 4633,   24, ...,    0,    0,    0],
       [ 189,   45,  229, ...,    0,    0,    0],
       [  40, 1751, 1620, ...,    0,    0,    0],
       ...,
       [2824, 2401,  709, ...,    0,    0,    0],
       [  78, 1145,   41, ...,    0,    0,    0],
       [   4,  209,   54, ...,    0,    0,    0]], dtype=int32)

In [18]:
train_x, valid_x, train_y,valid_y = train_test_split(train_seq_x, train_data['target'], test_size = 0.15, random_state = 42 )

In [19]:
### loading pretrained word-embeddings
embeddings_index = {}

for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding='utf-8')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

In [20]:
len(embeddings_index.keys())

999995

In [21]:
### token embedding mapping
embedding_matrix = np.zeros((len(word_index)+1, 300))

x = 0
unfound_words = []
for word, i in word_index.items():
    vec = embeddings_index.get(word)
    if vec is not None:
        x+=1
        embedding_matrix[i] = vec
    else:
        unfound_words.append(word)

print("Embeddings found for {} words out of {}".format(x, len(word_index)))

Embeddings found for 12084 words out of 22700


In [22]:
# unfound_words[400:450]

In [23]:
# embedding_matrix[-1]

In [24]:
# embedding_matrix.sum(axis=1)

### LSTM

In [25]:
from tensorflow.keras import layers

In [26]:
from tensorflow.keras import optimizers

In [27]:
from tensorflow.keras import models

In [31]:
def create_lstm_model():
    
    input_layer = layers.Input(shape=(70,)) # input layer
    embedding_layer = layers.Embedding(len(word_index)+1, 300, trainable = False, weights = [embedding_matrix])(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    lstm_layer = layers.LSTM(100 )(embedding_layer)
#     pooling_layer = layers.GlobalMaxPool1D()(lstm_layer)

#     output_layer0 = layers.Dense(128, activation = 'relu')(pooling_layer)
# #     output_layer0 = layers.BatchNormalization()(output_layer0)
#     output_layer0 = layers.Dropout(0.25)(output_layer0)

#     output_layer1 = layers.Dense(64, activation = 'relu')(output_layer0)
# #     output_layer1 = layers.BatchNormalization()(output_layer1)
#     output_layer1 = layers.Dropout(0.25)(output_layer1)
    
#     output_layer2 = layers.Dense(32, activation = 'relu')(output_layer1)
# #     output_layer2 = layers.BatchNormalization()(output_layer2)
#     output_layer2 = layers.Dropout(0.25)(output_layer2)
    
    output_layer3 = layers.Dense(16, activation = 'relu')(lstm_layer)
#     output_layer3 = layers.BatchNormalization()(output_layer3)
#     output_layer3 = layers.Dropout(0.25)(output_layer3)
    
    output_layer4 = layers.Dense(1, activation = 'sigmoid')(output_layer3)
    
    model= models.Model(inputs = input_layer, outputs = output_layer4)
    model.compile(optimizer = optimizers.Adam(learning_rate=0.0001) ,loss = "binary_crossentropy")
    
    return model

In [32]:
### training the model

model = create_lstm_model()


In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 70, 300)           6810300   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 70, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 16)                1616      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 6,972,333
Trainable params: 162,033
Non-trainable params: 6,810,300
_____________________________________________

In [34]:
train_model(model, train_x, train_y, valid_x, valid_y, 
            neural_network = True, epochs = 200,
            test_vectors = test_seq_x, submissions_data = sample_submission, submissions_file_prefix="lstm_submission" 
           )

Epoch 1/200
203/203 [==============================] - 14s 57ms/step - loss: 0.6795 - val_loss: 0.5207
Epoch 2/200
203/203 [==============================] - 12s 60ms/step - loss: 0.5015 - val_loss: 0.4538
Epoch 3/200
203/203 [==============================] - 10s 47ms/step - loss: 0.4644 - val_loss: 0.4311
Epoch 4/200
203/203 [==============================] - 12s 57ms/step - loss: 0.4491 - val_loss: 0.4269
Epoch 5/200
203/203 [==============================] - 13s 62ms/step - loss: 0.4437 - val_loss: 0.4211
Epoch 6/200
203/203 [==============================] - 12s 57ms/step - loss: 0.4400 - val_loss: 0.4281
Epoch 7/200
203/203 [==============================] - 11s 55ms/step - loss: 0.4340 - val_loss: 0.4412
Epoch 8/200
203/203 [==============================] - 9s 45ms/step - loss: 0.4453 - val_loss: 0.4186
Epoch 9/200
203/203 [==============================] - 10s 47ms/step - loss: 0.4294 - val_loss: 0.4168
Epoch 10/200
203/203 [==============================] - 12s 59ms/step - lo

### BiLSTM

In [35]:
def create_bilstm_model():
    
    input_layer = layers.Input(shape=(70,)) # input layer
    embedding_layer = layers.Embedding(len(word_index)+1, 300, trainable = False, weights = [embedding_matrix])(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    lstm_layer = layers.Bidirectional(layers.LSTM(100 ))(embedding_layer)
#     pooling_layer = layers.GlobalMaxPool1D()(lstm_layer)

#     output_layer0 = layers.Dense(128, activation = 'relu')(pooling_layer)
# #     output_layer0 = layers.BatchNormalization()(output_layer0)
#     output_layer0 = layers.Dropout(0.25)(output_layer0)

#     output_layer1 = layers.Dense(64, activation = 'relu')(output_layer0)
# #     output_layer1 = layers.BatchNormalization()(output_layer1)
#     output_layer1 = layers.Dropout(0.25)(output_layer1)
    
#     output_layer2 = layers.Dense(32, activation = 'relu')(output_layer1)
# #     output_layer2 = layers.BatchNormalization()(output_layer2)
#     output_layer2 = layers.Dropout(0.25)(output_layer2)
    
    output_layer3 = layers.Dense(16, activation = 'relu')(lstm_layer)
#     output_layer3 = layers.BatchNormalization()(output_layer3)
#     output_layer3 = layers.Dropout(0.25)(output_layer3)
    
    output_layer4 = layers.Dense(1, activation = 'sigmoid')(output_layer3)
    
    model= models.Model(inputs = input_layer, outputs = output_layer4)
    model.compile(optimizer = optimizers.Adam(learning_rate=0.0001) ,loss = "binary_crossentropy")
    
    return model

In [36]:
### training the model

model = create_bilstm_model()


In [37]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 70, 300)           6810300   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 70, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               320800    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                3216      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 7,134,333
Trainable params: 324,033
Non-trainable params: 6,810,300
___________________________________________

In [38]:
train_model(model, train_x, train_y, valid_x, valid_y, 
            neural_network = True, epochs = 200,
            test_vectors = test_seq_x, submissions_data = sample_submission, submissions_file_prefix="bilstm_submission" 
           )

Epoch 1/200
203/203 [==============================] - 23s 97ms/step - loss: 0.6748 - val_loss: 0.5269
Epoch 2/200
203/203 [==============================] - 20s 98ms/step - loss: 0.5056 - val_loss: 0.4405
Epoch 3/200
203/203 [==============================] - 19s 93ms/step - loss: 0.4687 - val_loss: 0.4429
Epoch 4/200
203/203 [==============================] - 16s 80ms/step - loss: 0.4368 - val_loss: 0.4634
Epoch 5/200
203/203 [==============================] - 16s 81ms/step - loss: 0.4607 - val_loss: 0.4202
Epoch 6/200
203/203 [==============================] - 16s 81ms/step - loss: 0.4270 - val_loss: 0.4285
Epoch 7/200
203/203 [==============================] - 17s 82ms/step - loss: 0.4269 - val_loss: 0.4203
Epoch 8/200
203/203 [==============================] - 17s 85ms/step - loss: 0.4136 - val_loss: 0.4206
Epoch 9/200
203/203 [==============================] - 17s 84ms/step - loss: 0.4300 - val_loss: 0.4150
Epoch 10/200
203/203 [==============================] - 17s 84ms/step - l